# Get all contact details of STB licensed tour guides

In [12]:
import os
import re, pickle, pdb
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [13]:
def get_contact_list(html_doc, verbose = 1):
    '''
    This function returns the contact list of a page of tour guides on the STB website, given the source represented as html_doc
    Input:
        html_doc (str): A string representing the entire page source of a page of tour guides on the STB website.
        verbose (bool): Prints status while executing if True, prints nothing if False.
    Output:
        contact_list (list of dict): A list of dictionaries, where each dictionary corresponds to the contact details (name/alias/tel/email) of an STB tour guide.
    '''
    x = re.findall('<a class="tooltip-icon".*?"(.*?)"|<div class="name long-truncated">(.*?)</div|<div class="alias-name">(.*?)</div',html_doc) # List of tuples of strings, of which at most one  is nonempty.
    print(f'First few elements of x:{x[:5]}')
    
    contact_list = []
    details = {'name':'','alias':'','tel':'','email':''}
    for elem in x: # elem is a tuple
        # Execute instructions based on which string in the tuple is nonempty. If all strings are empty then nothing is done.
        if elem[1]: # Then it must be a full name due to the format of the regex search (which starts a new entry in contact_list because all other details come after this.
            contact_list.append(details)
            details = {'name':'','alias':'','tel':'','email':''}
            details['name'] = elem[1].title() # Capitalize first letter of each word in name only.
        elif elem[2]: # Then it must be an alias due to the format of the regex search.
            details['alias'] = elem[2].title() # Capitalize first letter of each word in name only.
        elif elem[0]: # Then it can be either a telephone number or email due to the format of the regex search.
            if re.search('tel:',elem[0]):
                details['tel'] = re.findall('tel:(.*)',elem[0])[0]
            else:
                details['email'] = re.findall('mailto:(.*)',elem[0])[0]
    contact_list.append(details)
    contact_list.pop(0)
    return contact_list

Note to self: Somehow this code returns only 2940 results when the STB website claims to have 2962. Maybe there's something wrong with the code or the STB website, but 2940 email addresses is sufficient for our purpose so I'm just going to conveniently ignore it here. If there's a need to someone else can fix it next time.

In [16]:
contact_list = []
for pgno in tqdm(range(2962//15)): # Total entries//no. entries per pages
    try:
        url = f'https://trust.stb.gov.sg/site/content/tagaem/landing-page/tourist-guide.html?curIndex={pgno}' # Generate page url
        driver = webdriver.Firefox() # Open a new Firefox window. Requires gecko driver, see: https://selenium-python.readthedocs.io/installation.html
        driver.get(url) # Makes window go to the specified url
        html_doc = driver.page_source # Get the page source from the page.
        driver.close() # Close the Firefox window
        contact_list.extend(get_contact_list(html_doc)) # Extract names from page source
    except:
        print(f'Error in page {pgno}, moving on...') # Should not normally happen.
        continue
pd.DataFrame(contact_list).to_csv('stb-contact-details.csv')

  1%|▍                                                                               | 1/197 [01:08<3:44:04, 68.59s/it]

First few elements of x:[]


  1%|▊                                                                               | 2/197 [01:20<1:55:15, 35.46s/it]

First few elements of x:[('', 'LAU YUET NI', ''), ('', '', ''), ('tel:96960652', '', ''), ('mailto:lauyn@yahoo.com', '', ''), ('', '', '')]


  2%|█▏                                                                              | 3/197 [01:31<1:17:20, 23.92s/it]

...

First few elements of x:[('', 'Tee Bee Lock', ''), ('', '', 'Christine'), ('tel:98512358', '', ''), ('mailto:christinebl.tee@yahoo.com', '', ''), ('', '', 'Christine')]


 99%|███████████████████████████████████████████████████████████████████████████████▌| 196/197 [33:58<00:09,  9.91s/it]

First few elements of x:[('', 'Sun Tingting', ''), ('', '', ''), ('tel:82244112', '', ''), ('mailto:SUNTINGTING612@163.COM', '', ''), ('', '', '')]


100%|████████████████████████████████████████████████████████████████████████████████| 197/197 [34:08<00:00, 10.40s/it]

First few elements of x:[('', 'Zhang Junjuan', ''), ('', '', ''), ('tel:86930599', '', ''), ('mailto:junjuanzhang@gmail.com', '', ''), ('', '', '')]


# Generate all question text

In [17]:
rnos = [1,2,3,4,5,6]
rsts = ['Central Area (also known as the "Central Business District (CBD)")','East Region','Central Region','North Region','North-east Region','West Region']

pnos = ['a','b','c','d']
psts = ['full of life and exciting','chaotic and restless','calm and tranquil','lifeless and boring']

qnos = ['i','ii','iii','iv','v']

for rno, rst in zip(rnos,rsts): # Regions.
    for pno, pst in zip(pnos,psts): # Perceptual attributes.
        for idx, qno in enumerate(qnos): # Question numbers.
            qsts = [f'Q{rno})({pno})({qno}) Considering public open spaces (e.g. streets, squares, parks, etc.) in the {rst} of Singapore, where do you experience the soundscape to be the most {pst}? Please drag the marker on the map to that location (zoom in/out if needed). Please do not choose a location that you have never visited or passed by on foot before.',
            f'Q{rno})({pno})({qno}) Coordinates of your chosen location (please do not change these values; they will update when you drag the marker on the map):',
            f'Q{rno})({pno})({qno}) Please explain and elaborate on your choice of location in Q{rno})({pno})({qnos[0]}). For example, you can indicate the typical time and day of the week, etc. that you find the location to have a soundscape that is {pst}.',
            f'Q{rno})({pno})({qno}) How often do you visit your chosen location in Q{rno})({pno})({qnos[0]}), or pass by it on foot?',
            f'Q{rno})({pno})({qno}) How many times have you visited your chosen location in Q{rno})({pno})({qnos[0]}), or passed by it on foot? If you selected anything other than "Less often than once a year" for Q{rno})({pno})({qnos[3]}), then you should have visited your chosen location 10 or more times.',
            f'Q{rno})({pno})({qno}) On average, how long do you spend at your chosen location in Q{rno})({pno})({qnos[0]}) each time you visit it, or pass by it on foot? Please enter a time in minutes (1 hour = 60 minutes).']
            print(qsts[idx],end='\n\n')

Q1)(a)(i) Considering public open spaces (e.g. streets, squares, parks, etc.) in the Central Area (also known as the "Central Business District (CBD)") of Singapore, where do you experience the soundscape to be the most full of life and exciting? Please drag the marker on the map to that location (zoom in/out if needed). Please do not choose a location that you have never visited or passed by on foot before.

Q1)(a)(ii) Coordinates of your chosen location (please do not change these values; they will update when you drag the marker on the map):

Q1)(a)(iii) Please explain and elaborate on your choice of location in Q1)(a)(i). For example, you can indicate the typical time and day of the week, etc. that you find the location to have a soundscape that is full of life and exciting.

Q1)(a)(iv) How often do you visit your chosen location in Q1)(a)(i), or pass by it on foot?

Q1)(a)(v) How many times have you visited your chosen location in Q1)(a)(i), or passed by it on foot? If you selecte